Importando os pacotes

In [1]:
import pandas as pd

Carregando os dados

In [2]:
dados=pd.read_csv('dados2.csv')

In [3]:
# dados.iloc[:,[0,1,2]].rename(columns={'CONCLIENTE':'id'})
#cliente=dados.iloc[:,[0,1,2]]
cliente=dados[['CONCLIENTE','dataNascimentoCliente','sexoCliente']]
cliente.columns=['id','data_nascimento','sexo']
cliente=cliente[~cliente['id'].isna()]

In [4]:
inscricao=dados[['CONINSCRICAO','CONCLIENTE','CONPRODUTO','periodicidadeCobranca']]
inscricao.columns=['id','cliente_id','produto_id','periodicidade']
inscricao=inscricao[~inscricao['id'].isna()]

In [5]:
itemcontratado=dados[['CONITEMCONTRATADO','CONINSCRICAO','codigoSistemaItemContratado','dataInicioVigenciaCobertura','prazoCobertura','prazoContribuicao','prazoCertoRendaItemContratado','prazoMinimoGarantidoRendaItemContratado','prazoDecrescimoItemContratado','indiceReajusteItemContratado','valorCapitalSeguradoItemContratadoInicial']]
itemcontratado.columns=['id','inscricao_id','itemproduto_id','data_assinatura','prazo_cobertura','prazo_pagamento','prazo_renda','prazo_certo_renda','prazo_decrescimo','indice_reajuste','beneficio_inicial']
itemcontratado=itemcontratado[~itemcontratado['id'].isna()]

In [6]:
itembeneficiario=dados[['CONITEMCONTRATADOBENEFICIARIO','CONITEMCONTRATADO','CONBENEFICIARIO','percentualParticipacaoBeneficio']]
itembeneficiario.columns=['id','itemcontratado_id','beneficiario_id','percentual_beneficio']
itembeneficiario=itembeneficiario[~itembeneficiario['id'].isna()]

In [22]:
beneficiario=dados[['CONBENEFICIARIO','dataNascimentoBeneficiario','sexoBeneficiario']]
beneficiario.columns=['id','data_nascimento','sexo']
beneficiario=beneficiario[~beneficiario['id'].isna()]
beneficiario['sexo']=beneficiario['sexo'].fillna('NÃO INFORMADO')

In [23]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, inspect

In [24]:
engine=create_engine("sqlite:///dados.db")

In [25]:
cliente.to_sql('cliente',con=engine,if_exists="replace", index=False)
inscricao.to_sql('inscricao',con=engine,if_exists="replace", index=False)
itemcontratado.to_sql('itemcontratado',con=engine,if_exists="replace", index=False)
itembeneficiario.to_sql('itembeneficiario',con=engine,if_exists="replace", index=False)
beneficiario.to_sql('beneficiario',con=engine,if_exists="replace", index=False)

764

In [26]:
inspector = inspect(engine)
print(inspector.get_table_names())

['beneficiario', 'cliente', 'inscricao', 'itembeneficiario', 'itemcontratado']


In [27]:
cliente['sexo'].unique()

array(['M', 'F'], dtype=object)

In [28]:
Sexo = {
'F': 'FEMININO',
'M': 'MASCULINO'}

cliente['sexo_desc'] = cliente['sexo'].map(Sexo)
cliente.to_sql('cliente',con=engine,if_exists="replace", index=False)

1000

In [34]:
query = """SELECT B.SEXO, COUNT(*) AS CONTAGEM_BENEF, C.CONTAGEM_CLIENTE
            FROM BENEFICIARIO B
            LEFT JOIN (SELECT SEXO_DESC, COUNT(*) AS CONTAGEM_CLIENTE FROM CLIENTE
                        GROUP BY SEXO_DESC) C ON B.SEXO=C.SEXO_DESC
            GROUP BY B.SEXO
        """

In [35]:
pd.read_sql(query, engine)

,sexo,CONTAGEM_BENEF,CONTAGEM_CLIENTE
0,FEMININO,130,421.0
1,MASCULINO,432,579.0
2,NÃO INFORMADO,202,NaN


In [ ]:
query2 = """SELECT CLIENTE_ID, COUNT(*) AS CONTAGEM FROM INSCRICAO
            GROUP BY CLIENTE_ID
            HAVING CONTAGEM>1
            ORDER BY COUNT(*) DESC
        """
tabela=pd.read_sql(query2, engine)


In [ ]:
tabela.style.highlight_max(color= 'lightblue',subset='CONTAGEM').highlight_min(color = 'lightyellow',subset='CONTAGEM')

In [ ]:
query2 = """SELECT CLIENTE_ID, COUNT(*) AS CONTAGEM FROM INSCRICAO
            GROUP BY CLIENTE_ID
            HAVING CONTAGEM>1
            ORDER BY COUNT(*) DESC
        """
tabela=pd.read_sql(query2, engine)


In [77]:
query3 = """ 
SELECT DISTINCT A.BENEFICIARIO_ID,D.CLIENTE_ID FROM ITEMBENEFICIARIO A
INNER JOIN ITEMCONTRATADO B ON A.ITEMCONTRATADO_ID=B.ID
INNER JOIN INSCRICAO C ON B.INSCRICAO_ID=C.ID 
INNER JOIN 
(SELECT E.CLIENTE_ID,COUNT(*) AS CONTAGEM FROM 
(SELECT DISTINCT C.CLIENTE_ID, A.BENEFICIARIO_ID FROM ITEMBENEFICIARIO A
LEFT JOIN ITEMCONTRATADO B ON A.ITEMCONTRATADO_ID=B.ID
LEFT JOIN INSCRICAO C ON B.INSCRICAO_ID=C.ID) E
GROUP BY E.CLIENTE_ID
ORDER BY COUNT(*) DESC
LIMIT 2) D ON D.CLIENTE_ID=C.CLIENTE_ID
"""
pd.read_sql(query3, engine)


,beneficiario_id,CLIENTE_ID
0,3631309.0,7583711
1,3631310.0,7583711
2,3631311.0,7583711
3,3631312.0,7583711
4,3631313.0,7583711
5,3631314.0,7583711
6,3623815.0,1997789
7,3623816.0,1997789
8,3623817.0,1997789
9,3623818.0,1997789


In [84]:
query4 = """  select a.*, case when Quartile=1 then 'baixo'
                               when Quartile=2 then 'moderado'
                               else 'alto' end as classificacao from
(SELECT *, NTILE(3) OVER(ORDER BY BENEFICIO_INICIAL DESC) AS Quartile FROM ITEMCONTRATADO) a"""

tabela=pd.read_sql(query4, engine)
tabela['classificacao'].unique()

array(['baixo', 'moderado', 'alto'], dtype=object)

In [ ]:
POWER(POWER(B.BENEFICIO_INICIAL,2))/COUNT(*),0.5) AS DESVIO_P FROM

In [105]:
query4 = """ 
SELECT B.CLIENTE_ID, AVG(B.BENEFICIO_INICIAL) AS MEDIA_BENEF,  MAX(B.BENEFICIO_INICIAL) AS MAX_BENEF,
MIN(B.BENEFICIO_INICIAL) AS MIN_BENEF FROM
(SELECT C.CLIENTE_ID, B.BENEFICIO_INICIAL FROM ITEMCONTRATADO B
INNER JOIN INSCRICAO C ON B.INSCRICAO_ID=C.ID) B
GROUP BY B.CLIENTE_ID
"""
pd.read_sql(query4, engine)


,CLIENTE_ID,MEDIA_BENEF,MAX_BENEF,MIN_BENEF
0,144775,400000.00,400000.00,400000.00
1,168499,20000.00,20000.00,20000.00
2,206183,500000.00,500000.00,500000.00
3,208669,52885.48,52885.48,52885.48
4,235463,300000.00,300000.00,300000.00
...,...,...,...,...
630,7620078,450000.00,450000.00,450000.00
631,7620378,3000000.00,3000000.00,3000000.00
632,7621256,400000.00,400000.00,400000.00
633,7621715,500000.00,500000.00,500000.00


In [111]:
query5 = """ 
SELECT DISTINCT A.ID, A.DATA_NASCIMENTO,A.SEXO FROM
CLIENTE A INNER JOIN
(SELECT DISTINCT C.CLIENTE_ID, C.PERIODICIDADE, B.DATA_ASSINATURA FROM ITEMCONTRATADO B
INNER JOIN INSCRICAO C ON B.INSCRICAO_ID=C.ID
WHERE C.PERIODICIDADE = 'MENSAL' OR (B.DATA_ASSINATURA>='2022-01-06' AND B.DATA_ASSINATURA<='2022-01-10')
) B ON B.CLIENTE_ID=A.ID
"""
pd.read_sql(query5, engine)


,id,data_nascimento,sexo
0,1750926,1977-10-06,M
1,4300961,1957-04-15,M
2,7488418,1963-03-05,F
3,2700079,1966-05-20,F
4,7503154,1983-03-04,F
...,...,...,...
105,7598207,1988-12-09,F
106,7598014,1970-10-06,M
107,7598746,1990-03-29,M
108,2435127,1977-06-17,M
